In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from transformers import AutoModel, BertTokenizer, BertForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm

In [2]:
max_length = 128
path = os.getcwd()
# path = os.path.join(path, 'train_data')
path

'/content'

In [55]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence", num_labels=2, output_attentions = False, output_hidden_states = False)
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
model.to('cuda')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased-sentence and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [8]:
# from safetensors import safe_open
# tensors = {}
# with safe_open("/content/results/checkpoint-604/model.safetensors", framework="pt", device=0) as f:
#     for k in f.keys():
#         tensors[k] = f.get_tensor(k)
# model.load_state_dict(tensors)

In [38]:
positive = 0
negative = 0
lines = []
labels = []
fonts = []
bolds = []
for output_file in os.listdir(path):
    file_dir = os.path.join(path, output_file)
    if not file_dir.endswith('.txt'):
        continue
    with open(file_dir, 'r', encoding='UTF-8') as f:
        for line in f.readlines():
            flag = 0
            line1, line2 = line.split('[font=')
            lines.append(line1[:-1])
            if line2[-3] == '1':
                positive += 1
                labels.append(1)

                flag = 1
            else:
                negative += 1
                labels.append(0)
            line2 = line2.split(', ')
            fonts.append(float(line2[0]))
            bolds.append(int(line2[1][-1]))

In [39]:
negative

62740

In [42]:
dataset = pd.DataFrame({
    'target': labels,
    'text': lines,
})
dataset[dataset.target == 0] = dataset[(dataset.target == 0) & (dataset.text.apply(lambda x: len(x.split()) >= 2))]
dataset = dataset.dropna()
dataset.reset_index(drop=True, inplace=True)
def add_context(df):
    # df['prev_text'] = df['text'].shift(1).fillna('')
    df['next_text'] = df['text'].shift(-1).fillna('')
    df['context'] = df['next_text']
    # df['combined'] = '[CLS] ' + df['prev_text'] + ' [SEP] '+ df['text'] + ' [SEP] ' + df['context'] + ' [SEP]'
    df['combined'] = '[CLS] ' + df['text'] + ' [SEP] ' + df['context']
    return df

dataset = add_context(dataset)
# dataset.drop(columns=['next_text', 'prev_text', 'context'], inplace=True)
dataset.drop(columns=['next_text', 'context'], inplace=True)

df_target_0 = dataset[dataset['target'] == 0]

df_target_0_sampled = df_target_0.sample(frac=0.7, random_state=42)

# Drop the sampled rows from the original DataFrame
dataset = dataset.drop(df_target_0_sampled.index)
dataset.reset_index(drop=True, inplace=True)

<ipython-input-42-bacd0f33a0f4>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['next_text'] = df['text'].shift(-1).fillna('')
<ipython-input-42-bacd0f33a0f4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['context'] = df['next_text']
<ipython-input-42-bacd0f33a0f4>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [43]:
dataset[dataset.target==0]

,target,text,combined
3,0.0,«Арсагера» на заседании Совета директоров 27 а...,[CLS] «Арсагера» на заседании Совета директоро...
4,0.0,М Принципы корпоративного управления Критерии ...,[CLS] М Принципы корпоративного управления Кри...
5,0.0,1.1 | Общество должно обеспечивать равное и сп...,[CLS] 1.1 | Общество должно обеспечивать равно...
6,0.0,1.1.1 | Общество создает для акционеров 1. Общ...,[CLS] 1.1.1 | Общество создает для акционеров ...
7,0.0,"для участия в общем собрании, условия |такой к...","[CLS] для участия в общем собрании, условия |т..."
...,...,...,...
18090,0.0,Представитель подрядчика /,[CLS] Представитель подрядчика / [SEP] Предста...
18091,0.0,Представитель общественной,[CLS] Представитель общественной [SEP] Предста...
18093,0.0,Тел.: +7 (499) 949-29-79,[CLS] Тел.: +7 (499) 949-29-79 [SEP] Карева Ви...
18094,0.0,проектов и сделок с акционерным капиталом,[CLS] проектов и сделок с акционерным капитало...


In [44]:
dataset[dataset.target==1]

,target,text,combined
0,1.0,ОТЧЕТ,[CLS] ОТЧЕТ [SEP] о соблюдении принципов и рек...
1,1.0,о соблюдении принципов и рекомендаций Кодекса,[CLS] о соблюдении принципов и рекомендаций Ко...
2,1.0,корпоративного управления,[CLS] корпоративного управления [SEP] Настоящи...
582,1.0,Введение,"[CLS] Введение [SEP] Информация, содержащаяся ..."
599,1.0,Раздел 1. Управленческий отчет эмитента,[CLS] Раздел 1. Управленческий отчет эмитента ...
...,...,...,...
18033,1.0,составленная на основе консолидированной,[CLS] составленная на основе консолидированной...
18034,1.0,"финансовой отчетности, за год, закончившийся 3...","[CLS] финансовой отчетности, за год, закончивш..."
18035,1.0,аудиторское заключение независимых аудиторов,[CLS] аудиторское заключение независимых аудит...
18082,1.0,Анкета обратной связи,[CLS] Анкета обратной связи [SEP] Уважаемые чи...


In [45]:
from sklearn.utils import resample

In [47]:
df_minority = dataset[dataset['target'] == 1]
df_minority_oversampled = resample(df_minority,
                                   replace=True,
                                   n_samples=6000,
                                   random_state=42)
df_oversampled = pd.concat([dataset[dataset['target'] == 0], df_minority_oversampled])

In [48]:
df_oversampled[df_oversampled['target'] == 1]

,target,text,combined
1543,1.0,3.6. — Запасы. Учетная политика,[CLS] 3.6. — Запасы. Учетная политика [SEP] За...
13165,1.0,ОБЩЕСТВА В ОБЛАСТИ ВОЗНАГРАЖДЕНИЯ И (ИЛИ) КОМП...,[CLS] ОБЩЕСТВА В ОБЛАСТИ ВОЗНАГРАЖДЕНИЯ И (ИЛИ...
4237,1.0,10. Сопроводительная информация к статьям отче...,[CLS] 10. Сопроводительная информация к статья...
1611,1.0,3.10. Кредиторская задолженность,[CLS] 3.10. Кредиторская задолженность [SEP] Д...
1322,1.0,Отчет о финансовых результатах,[CLS] Отчет о финансовых результатах [SEP] за ...
...,...,...,...
13552,1.0,Примечание 3. Основы составления отчетности,[CLS] Примечание 3. Основы составления отчетно...
16085,1.0,22. Платежи в погашение обязательств по аренде,[CLS] 22. Платежи в погашение обязательств по ...
13941,1.0,"Примечание 6. Финансовые активы, в обязательно...","[CLS] Примечание 6. Финансовые активы, в обяза..."
1270,1.0,Не указывается в настоящем отчете,[CLS] Не указывается в настоящем отчете [SEP] ...


In [49]:
train_df, test_df = train_test_split(df_oversampled, test_size=0.3, random_state=42, shuffle=True)
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
train_df

,target,text,combined
0,0.0,экономическое содержание таких взаимоотношений...,[CLS] экономическое содержание таких взаимоотн...
1,0.0,кросс-платформенное приложение собственной разра-,[CLS] кросс-платформенное приложение собственн...
2,0.0,В случае непредставления или несвоевременного ...,[CLS] В случае непредставления или несвоевреме...
3,0.0,банка Российской Федерации,[CLS] банка Российской Федерации [SEP] 16 Сред...
4,0.0,(выпущены 23 января 2020 г. и вступают в силу ...,[CLS] (выпущены 23 января 2020 г. и вступают в...
...,...,...,...
16403,0.0,Не позднее 1 (Одного) дня с даты опубликования...,[CLS] Не позднее 1 (Одного) дня с даты опублик...
16404,1.0,за Январь - Декабрь 2023 г. Коды,[CLS] за Январь - Декабрь 2023 г. Коды [SEP] Ф...
16405,0.0,"мультибрендовое предложение, позволяющее","[CLS] мультибрендовое предложение, позволяющее..."
16406,0.0,действия лицензий (в случае если лицензия выда...,[CLS] действия лицензий (в случае если лицензи...


In [53]:
print(len([string for string in train_df.combined.values if len(string.split()) > 50]))

7


In [51]:
train_text = train_df.combined.astype('str')
train_labels = train_df.target
test_text = test_df.combined.astype('str')
test_labels = test_df.target

In [52]:
test_text

,combined
0,"[CLS] СВЕДЕНИЯ ОБ ОБЯЗАТЕЛЬНЫХ НОРМАТИВАХ, НОР..."
1,[CLS] капитал общества [SEP] Принадлежит к той...
2,[CLS] 8.8. — Изменение резерва на возможные по...
3,[CLS] классифицируемыми как оцениваемые по спр...
4,[CLS] информации для подготовки последующих до...
...,...
7027,"[CLS] принимающих риски, и учет уровня принима..."
7028,[CLS] и внутренних пассажирских и грузовых воз...
7029,[CLS] прочие активы [SEP] Группы 2.1 и5.1
7030,[CLS] исправления и порядок раскрытия информац...


In [54]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.values,
    max_length = 50,
    padding = 'max_length',
    truncation = True
)
tokens_test = tokenizer.batch_encode_plus(
    test_text.values,
    max_length = 50,
    padding = 'max_length',
    truncation = True
)

In [56]:
class Data(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        if self.labels[idx] == 1.0:
            item["labels"] = torch.tensor([0.0, self.labels[idx]])
        else:
            item["labels"] = torch.tensor([1.0, 0.0])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = Data(tokens_train, train_labels)
test_dataset = Data(tokens_test, test_labels)

In [57]:
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights

array([0.6703162, 1.967858 ])

In [58]:
class CustomTrainer(Trainer):
    def __init__(self, model, *args, cw=None, **kwargs):
        super().__init__(model, *args, **kwargs)
        self.cw = torch.tensor(cw)

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        if self.cw is not None:
            self.cw = self.cw.to('cuda')
            loss_fct = nn.CrossEntropyLoss(weight=self.cw)
        else:
            loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [59]:
torch.cuda.empty_cache()

In [60]:
training_args = TrainingArguments(
    output_dir = './results', #Выходной каталог
    num_train_epochs = 15, #Кол-во эпох для обучения
    per_device_train_batch_size = 128, #Размер пакета для каждого устройства во время обучения
    per_device_eval_batch_size = 128, #Размер пакета для каждого устройства во время валидации
    weight_decay =0.01, #Понижение весов
    logging_dir = './logs', #Каталог для хранения журналов
    load_best_model_at_end = True, #Загружать ли лучшую модель после обучения
    learning_rate = 1e-5, #Скорость обучения
    eval_strategy ='epoch', #Валидация после каждой эпохи (можно сделать после конкретного кол-ва шагов)
    logging_strategy = 'epoch', #Логирование после каждой эпохи
    save_strategy = 'epoch', #Сохранение после каждой эпохи
    save_total_limit = 5,
    seed=21)

trainer = CustomTrainer(model=model,
                  tokenizer = tokenizer,
                  args = training_args,
                  train_dataset = train_dataset,
                  eval_dataset = test_dataset,
                  cw = class_weights)

In [61]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.247500,0.085738
2,0.061500,0.060041
3,0.031800,0.031412
4,0.023300,0.066288


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'bert_next_context.pt')

In [63]:
def add_context(df):
    #df['prev_text'] = df['text'].shift(1).fillna('')
    df['next_text'] = df['text'].shift(-1).fillna('')
    df['context'] = df['next_text']
    df['combined'] = '[CLS] ' + df['text'] + ' [SEP] ' + df['context']
    return df

# Read and process data
ans = []
with open('output51.txt', 'r') as f:
    data = []
    max_length = 0
    for line in f.readlines():
        line, _ = line.split('[font=')
        max_length = max(len(line), max_length)
        data.append(line)
    data = pd.DataFrame(data, columns=['text'])
    data = add_context(data)

# Define batch size
batch_size = 32

# Process data in batches
model.to('cuda')
for start_idx in range(0, len(data), batch_size):
    end_idx = min(start_idx + batch_size, len(data))
    batch_data = data[start_idx:end_idx]

    inputs = tokenizer.batch_encode_plus(
        batch_data['combined'].tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    with torch.no_grad():
        outputs = model(**inputs.to('cuda'))
        preds = torch.softmax(outputs.logits, dim=-1)
        for idx, pred in enumerate(preds):
            if pred[1] > 0.99:
                out = tokenizer.decode(inputs['input_ids'][idx].tolist())
                out = out.split('[SEP]')[1]
                print(out)

            if len(ans) == 30:
                break
    if len(ans) == 30:
        break

 Акционерам публичного акционерного общества « Авиапионный комплекс им. 
 Мы провели аудит прилагаемой годовой бухгалтерской ( финансовой ) отчетности 
 годовую бухгалтерскую ( фипапсовую ) отчетность 
 Руководства несет ответственность за подготовку и достоверное представлепие 
 на 31 декабрн 2023 г. 
 11. КАПИТАЛ И РЕЗЕРВЫ 
 за Январь - Декабрь 2023 г. Коды 
 Форма по ОКУД 0710002 
 за Январь - Декабрь 2023 г. Коды 
 Форма по ОКУД ] 0710004 
 за Январь - Декабрь 2023 г. Коды 
 и отчету о финансовых результатах ( тыс. руб. ) 
 На начало года Изменения за период На конец перусда 
 ` Изменения за период 
 4. 1 _ Наличие и движение запасов 
 Е Е ЕВ ЕЕ ШЕЕ МЫ ЕТЗЯЕЕ МИЕНЕТЕЕЕЕ ИЕБЕЕЗЕТЕ ВИС ЕСТ ВИ ЗЕНА ВЕЕТ ЕБЕИЯ 
 Наименование показателя Код ` На31 декабря 2653 т. На 31 декабря 2127 г На ЭТ декабря 2021 г 
 3. 1. Напичие и движение финансовых вложений 
 На начало года 
 Наименование показателя 
 2. 1. Наличие и движение основных средств 
 Ив насало гспа 
 Изменения за период, 
 5. 1. Нал

In [37]:
torch.cuda.empty_cache()

# ВСЯКИЕ ЭКСПЕРИМЕНТЫ НИЖЕ

In [ ]:
"4.1_Наличие и движение запасов".replace("_", ' ')

'4.1 Наличие и движение запасов'

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-multitask")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-multitask")

input_text = "4.1_Наличие и движение запасов"

input_text = f"headline |  {input_text}"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

outputs = model.generate(input_ids)

corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Corrected text: {corrected_text}")


Corrected text: Эксперт:


In [ ]:
model.to('cuda')
ans = []
with open('texts_normal/output51.txt', 'r', encoding='UTF-8') as f:
    i = 0
    for line in f.readlines():
        line, _ = line.split('[font=')
        line = line.replace('_', ' ')
        input = tokenizer(line, return_tensors='pt', truncation=True, padding=True).to('cuda')
        with torch.no_grad():
            outputs = model(**input)
            preds = torch.softmax(outputs.logits, dim=-1)
            if preds[0][1] > 0.99:
                i += 1
                print(tokenizer.decode(input['input_ids'][0].tolist()))
                ans.append(tokenizer.decode(input['input_ids'][0].tolist()))
        if i == 50:
            break

[CLS] АУДИТОРСКОЕ ЗАКЛЮЧЕНИЕ НЕЗАВИСИМОГО АУДИТОРА [SEP]
[CLS] Мнение [SEP]
[CLS] АУДИТОР [SEP]
[CLS] Ключевые вопросы аудита [SEP]
[CLS] Ответственность руководства и членов совета директоров аудируемого лица за [SEP]
[CLS] деятельности. [SEP]
[CLS] аудируемого лица ; [SEP]
[CLS] АУДИТОР [SEP]
[CLS] Бухгалтерский баланс [SEP]
[CLS] Кады [SEP]
[CLS] АКТИВ [SEP]
[CLS] 1. ВНЕОБОРОТНЫЕ АКТИВЫ [SEP]
[CLS] 1. ОБОРОТНЫЕ АКТИВЫ [SEP]
[CLS] ПАССИВ [SEP]
[CLS] 11. КАПИТАЛ И РЕЗЕРВЫ [SEP]
[CLS] Отчет о финансовых результатах [SEP]
[CLS] за Январь - Декабрь 2023 г. Коды [SEP]
[CLS] Отчет об изменениях капитала [SEP]
[CLS] за Январь - Декабрь 2023 г. Коды [SEP]
[CLS] 1. Движение капитала [SEP]
[CLS]. 2. Корректировки в связи с изменением учетной политики и исправлением ошибок [SEP]
[CLS] 3. Чистые активы [SEP]
[CLS] Отчет о движении денежных средств [SEP]
[CLS] за Январь - Декабрь 2023 г. Коды [SEP]
[CLS] Пояснения к бухгалтерскому балансу [SEP]
[CLS] 4. 1. Наличие и движение нематериальных активо

In [ ]:
torch.softmax(model(**tokenizer("4.1 Наличие и движение запасов", return_tensors='pt', truncation=True).to('cuda')).logits, dim=-1)[0][1]

tensor(0.9964, device='cuda:0', grad_fn=<SelectBackward0>)

In [ ]:
ans = [ l[5:-5] for l in ans]
ans

[' АУДИТОРСКОЕ ЗАКЛЮЧЕНИЕ НЕЗАВИСИМОГО АУДИТОРА ',
 ' Мнение ',
 ' АУДИТОР ',
 ' Ключевые вопросы аудита ',
 ' Ответственность руководства и членов совета директоров аудируемого лица за ',
 ' деятельности. ',
 ' аудируемого лица ; ',
 ' АУДИТОР ',
 ' Бухгалтерский баланс ',
 ' Кады ',
 ' АКТИВ ',
 ' 1. ВНЕОБОРОТНЫЕ АКТИВЫ ',
 ' 1. ОБОРОТНЫЕ АКТИВЫ ',
 ' ПАССИВ ',
 ' 11. КАПИТАЛ И РЕЗЕРВЫ ',
 ' Отчет о финансовых результатах ',
 ' за Январь - Декабрь 2023 г. Коды ',
 ' Отчет об изменениях капитала ',
 ' за Январь - Декабрь 2023 г. Коды ',
 ' 1. Движение капитала ',
 '. 2. Корректировки в связи с изменением учетной политики и исправлением ошибок ',
 ' 3. Чистые активы ',
 ' Отчет о движении денежных средств ',
 ' за Январь - Декабрь 2023 г. Коды ',
 ' Пояснения к бухгалтерскому балансу ',
 ' 4. 1. Наличие и движение нематериальных активов ',
 ' 1. 3. Нематериальные активы с полностью погашенной стоимостью ',
 ' 1. 4. Наличие и движение результатов НИОКР. ',
 ' 1. 5. Незаконченные и неофо

In [ ]:
for i in range(len(ans)-1, 0, -1):
    str1 = ans[i-1]
    str2 = ans[i]
    line = str1 + str2
    inputs = tokenizer(line, return_tensors='pt', truncation=True, padding=True, max_length=64).to('cuda')
    with torch.no_grad():
        outputs = model(**inputs)
        preds = torch.softmax(outputs.logits, dim=-1)
        if preds[0][1] > 0.995:
            ans[i-1] = line
            ans[i] = ''
final = [l for l in ans if len(l) != 0]

In [ ]:
final

[' АУДИТОРСКОЕ ЗАКЛЮЧЕНИЕ НЕЗАВИСИМОГО АУДИТОРА ',
 ' Мнение  АУДИТОР ',
 ' Ключевые вопросы аудита ',
 ' Ответственность руководства и членов совета директоров аудируемого лица за ',
 ' деятельности. ',
 ' аудируемого лица ; ',
 ' АУДИТОР  Бухгалтерский баланс ',
 ' Кады  АКТИВ ',
 ' 1. ВНЕОБОРОТНЫЕ АКТИВЫ ',
 ' 1. ОБОРОТНЫЕ АКТИВЫ  ПАССИВ ',
 ' 11. КАПИТАЛ И РЕЗЕРВЫ  Отчет о финансовых результатах  за Январь - Декабрь 2023 г. Коды ',
 ' Отчет об изменениях капитала  за Январь - Декабрь 2023 г. Коды  1. Движение капитала ',
 '. 2. Корректировки в связи с изменением учетной политики и исправлением ошибок ',
 ' 3. Чистые активы ',
 ' Отчет о движении денежных средств  за Январь - Декабрь 2023 г. Коды  Пояснения к бухгалтерскому балансу ',
 ' 4. 1. Наличие и движение нематериальных активов ',
 ' 1. 3. Нематериальные активы с полностью погашенной стоимостью ',
 ' 1. 4. Наличие и движение результатов НИОКР. ',
 ' 1. 5. Незаконченные и неоформленные НИОКР и незаконченные операции по приобре

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
tmp = ans.copy()

In [ ]:
trans = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
for i in range(len(ans)-1, 0, -1):
    str1 = tmp[i-1]
    str2 = tmp[i]
    line = str1 + str2
    emb1 = trans.encode(str1)
    emb2 = trans.encode(str2)
    if util.pytorch_cos_sim(emb1, emb2) > 0.8:
        print(str1+str2)


 1. ВНЕОБОРОТНЫЕ АКТИВЫ  1. ОБОРОТНЫЕ АКТИВЫ 


In [ ]:
from transformers import AutoModelForSeq2SeqLM, T5TokenizerFast

MODEL_NAME = 'UrukHan/t5-russian-spell'
MAX_INPUT = 256

tokenizer = T5TokenizerFast.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

input_sequences = ['Оглавление', 'Информация об Отчете 5', 'История создания и развития АО « Атомнергопром ний', 'Глава 2. СТРАТЕГИЯ ДЕЯТЕЛЬНОСТИ нони', 'Глава 3. Результаты межлунаролной деятельности.', 'Глава 6. Цифровая трансформация.', '" Глава 7, Система управления', '7.. Корпоративное управление. 2', 'Глава 1. АО « Атомонергопром » сегодня', '1. иформация об акционерах АО « Атомэнергопром »', 'Информация об Отчете раскрытия в Отчет', 'Верификация отчетной информации.', 'Обращение руководства', 'АО « Атомэнергопром » сегодня.', 'История создания и развития АО « Атомэнергопром »', 'Основные события 2020 года', 'Глава 2. Стратегия леятельности', '2. 1. Стратегия деятельности до 2030 года', '2. 2. Управление устойчивым развитием управления', '« Диверсификация бизнеса »', 'Глава 3. Результаты межлународной леятельности', 'Глава 4. Результаты ливизионов.', '4. 1. Горнорудный дивизион', 'Новые продукты', '4. 2. Топливный дивизион.', '« Лидер ». продукции', '4. 3. Машиностроительный дивизион', '№3.', 'ЕРС.', '4. 5. Электроэнергетический ливизион', 'Глава 5. Инновации и новые продукты', '5. 1. Наука и инновации обязательств', '5 - 2. Диверсификация бизнеса', 'Композитные материалы', 'Промышленное оборудование', 'Глава 6. Цифровая трансформация', '6. 1. Единая пифровая стратегия', '6. 2. Участие в иифровизации России', '6. 3. Цифровые продукты', '6. 4. Внутренняя пифровизация', '6. 5. Планы на 2021 год', 'Глава 7. Система управления', '7л. Корпоративное управление', 'Органы управления', 'Комиров', 'Отчет совета директоров', 'АО « Атомэнергопром »,', 'АО « Атомэнергопром ». инструмент', '10. Риск утраты и нансниыя', 'Страхование рисков', '7. 3. Управление финансовой леятельностью', '7. 4. Управление инвестиционной леятельностью. правления', '7. 5. Система внутреннего контроля.', 'Глава 8. Кадровая политика и социальная ответственность.', '1. Работа с персоналом. правка :', '« Лидеры России »', 'Прочее', 'Глава 9. Безопасность деятельности.', '9. 1. Ядерная и радиационная безопасность и охрана труда структурных подразделений. функциональной подсистемы.', '9. 2. Экологическая безопасность холодильного оборудования', 'АО « Лунное »', 'Приложения', 'Приложение', 'Диверсия бизнеса', 'Корпоративное управление продукты', 'Наставничество » виток', 'Приложение 2. Обобщенная консолилированная финансовая отчетность аудиторское заключение независимых аудиторов', 'РОДА "', 'Анкета обратной связи', 'Представитель самоуправления', 'Представитель', 'Представитель СМИ. сообщества', 'Контактная информация', 'Казначейство']   # или можно использовать одиночные фразы:  input_sequences = 'сеглдыя хорош ден'

task_prefix = "Spell correct: "
if type(input_sequences) != list: input_sequences = [input_sequences]
encoded = tokenizer(
  [task_prefix + sequence for sequence in input_sequences],
  padding="longest",
  max_length=MAX_INPUT,
  truncation=True,
  return_tensors="pt",
).to('cuda')

predicts = model.generate(**encoded, )

tokenizer.batch_decode(predicts, skip_special_tokens=True)

['Оглавление',
 'Информация об отчете 5.',
 'История создания и развития АО «Атомэнергопромний»',
 'Глава 2. Стратегия деятельности НОН.',
 'Глава 3. Результаты межлунной деятельности.',
 'Глава 6. Цифровая трансформация.',
 '" Глава 7, Система управления.',
 '7. Корпоративное управление. 2.',
 'Глава 1. АО «Атомэнергопром» сегодня.',
 '1. Информация об акционерах АО «Атомэнергопром».',
 'Информация об отчете раскрыта в отчете',
 'Верификация отчетной информации.',
 'Обращение руководства',
 'АО «Атомэнергопром» сегодня.',
 'История создания и развития АО «Атомэнергопром»',
 'Основные события 2020 года',
 'Глава 2. Стратегия деятельности.',
 '2. 1. Стратегия деятельности до 2030 года.',
 '2. 2. Управление устойчивым развитием управления.',
 '. . . Диверсификация бизнеса».',
 'Глава 3. Результаты международной деятельности.',
 'Глава 4. Результаты дивизионов.',
 '4. 1. Горнорудный дивизион.',
 'Новые продукты',
 '4. 2. Топливный дивизион.',
 '. . . . . . . . . . . . . . . . . . . . . . 

In [ ]:
torch.cuda.is_available()

True